# Bespokefit workshop live session

## Learning objectives

By the end of this workshop you will be able to:

- 1. Build, configure and save a general bespokefit optimisation workflow
- 2. Build molecule specific optimizaion schemas from the general workflow
- 3. Load QCArchive computed data using OpenFF-QCSubmit as a fitting reference
- 4. Optimize some bespoke torsion parameters
- 5. analysis the results from ForceBalance 
- 6. Generate refernce data locally using xtb on the fly

In [1]:
from openff.qcsubmit.results import TorsionDriveResultCollection
from openff.bespokefit.workflows import BespokeWorkflowFactory
from openff.bespokefit.schema.optimizers import ForceBalanceSchema
from openff.bespokefit.schema.targets import TorsionProfileTargetSchema
from openff.bespokefit.bespoke import Executor
from openff.bespokefit.schema.data import BespokeQCData
from openff.qcsubmit.common_structures import QCSpec
from openff.toolkit.topology import Molecule
from qcportal import FractalClient
from pprint import pprint

## 1. Building the general workflow

Bespokefit aims to provide a reproducable parameter optimization workflow for SMIRNOFF based force fields. As such normal bespokefit execution starts with a general fitting workflow. This captures every process in the workflow along with any ajustable settings such as how the reference data should be generated. Lets start with a basic workflow which should be ready to use for torsion fitting.

In [2]:
# start with an empty workflow 
workflow = BespokeWorkflowFactory(fragmentation_engine=None, parameter_settings=[], target_smirks=[], target_templates=[])
pprint(workflow.dict())

{'expand_torsion_terms': True,
 'fragmentation_engine': None,
 'generate_bespoke_terms': True,
 'initial_force_field': 'openff_unconstrained-1.3.0.offxml',
 'optimizer': {'adaptive_damping': 1.0,
               'adaptive_factor': 0.2,
               'eigenvalue_lower_bound': 0.01,
               'error_tolerance': 1.0,
               'extras': {},
               'finite_difference_h': 0.01,
               'gradient_convergence_threshold': 0.01,
               'initial_trust_radius': -0.25,
               'job_type': 'optimize',
               'max_iterations': 10,
               'minimum_trust_radius': 0.05,
               'n_criteria': 2,
               'normalize_weights': False,
               'objective_convergence_threshold': 0.01,
               'penalty_additive': 1.0,
               'penalty_type': 'L1',
               'step_convergence_threshold': 0.01,
               'type': 'ForceBalance'},
 'parameter_settings': [],
 'target_smirks': [],
 'target_templates': []}


## 2. Build molecule specific schema



In [3]:
# load the molecules
mol = Molecule.from_file("data/bace.sdf")

OSError: File error: Bad input file data/bace.sdf

In [ ]:
mol[5]

In [ ]:
# load the default factory 
workflow = BespokeWorkflowFactory()

In [ ]:
# process all molecules
schema = workflow.optimization_schemas_from_molecules(mol[5])

In [ ]:
# pull out the new smirks and show how they transfer between the parent and the fragment

In [ ]:
# look into the data in the fitting schema
schema[0].targets[0].reference_data.dict(exclude={"tasks"})

## 3. Loading data from QCArchive

In [ ]:
client = FractalClient()

In [ ]:
client.list_collections("torsiondrivedataset")

In [ ]:
# create a result from the dataset we know our molecule is in
result = TorsionDriveResultCollection.from_server(client, "OpenFF-benchmark-ligand-fragments-v1.0", "default")

In [ ]:
# check how many results we have
result.n_molecules

In [ ]:
result.n_results

So we have 368 unique molecules and 481 torsiondrives, with some molecules have multipule torsion driven. All scans are 1D however.

In [ ]:
records_and_molecules = result.to_records()

In [ ]:
# show what we have pulled down
record, torsion_molecule = records_and_molecules[0]

In [ ]:
pprint(record.dict())

In [ ]:
torsion_molecule

In [ ]:
for molecule in schema:
    molecule.update_with_results(records_and_molecules)

In [ ]:
for molecule in schema:
    print(molecule.ready_for_fitting)

## 4. Optimize a bespoke torsion parameter

In [ ]:
executor = Executor()

In [ ]:
executor.execute(schema[0])